In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import bs4

In [2]:
def total_data_data_cleansing (raw_data):
    new_list = []
    for i in range(len(raw_data)) :
        data = raw_data[i]
        frist_step = data.replace('ขายแล้ว','')
        second_step = frist_step.replace('ชิ้น','')
        thrid_step = second_step.replace('พัน','000')
        fourth_step =thrid_step.replace('.','')
        new_list.append(fourth_step)
    return new_list #เฉพาะยอดขาย

def price_data_data_cleansing (raw_data):
    new_list = []
    for i in range(len(raw_data)) :
        data = raw_data[i]
        frist_step = data.replace('฿','')
        new_list.append(frist_step)
    return new_list #เฉพาะราคา

def number_of_page (text):
    first = text[0].replace('1/','')
    secound = int(first)
    return secound #หาจำนวนหน้า

def add_to_list_soup(raw_data):
    list_raw_data = []
    for i in raw_data:
        list_raw_data.append(i.text)
    return list_raw_data

def append_to_list (first_list,second_list):
    for i in range(len(second_list)):
        first_list.append(second_list[i])

In [3]:
list_title_data = []
list_clean_price_data = []
list_clean_total_data = []

total_class = 'r6HknA' #ยอดขาย
price_class = 'vioxXd rVLWG6' #ราคา
title_class = 'FDn--+' #คำอธิบายรายการสินค้า
all_of_page_class = 'shopee-mini-page-controller__state' #หาจำนวนหน้า


In [4]:
product = str(input()) #สินค้าที่ต้องการ

เสื้อผู้ชาย


In [6]:
driver = webdriver.Chrome()
website = 'https://shopee.co.th/search?keyword='+product+'&page='
driver.get(website)
time.sleep(1)
thai_button = driver.find_element(by=By.XPATH, value='/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

driver.execute_script("document.body.style.zoom='10%'")
time.sleep(1)
driver.execute_script("document.body.style.zoom='100%'")
data = driver.page_source

soup = bs4.BeautifulSoup(data)

all_of_page_data = soup.find_all('div',{'class':all_of_page_class})

clean_all_of_page_class_data = number_of_page(add_to_list_soup(all_of_page_data))

In [8]:
for page in  range(0,clean_all_of_page_class_data) :
    website = 'https://shopee.co.th/search?keyword='+product+'&page='+str(page)
    driver.get(website)
    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(2)
    driver.execute_script("document.body.style.zoom='100%'")
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    price_data = soup.find_all('div',{'class':price_class})
    total_data = soup.find_all('div',{'class':total_class})
    title_data = soup.find_all('div',{'class':title_class})
    clean_title_data = add_to_list_soup(title_data)
    clean_price_data= price_data_data_cleansing(add_to_list_soup(price_data))
    clean_total_data = total_data_data_cleansing(add_to_list_soup(total_data))
    append_to_list(list_title_data,clean_title_data)
    append_to_list(list_clean_price_data,clean_price_data)
    append_to_list(list_clean_total_data,clean_total_data)
table_1 =  pd.DataFrame([list_title_data,list_clean_price_data,list_clean_total_data])
table_2 = table_1.transpose()
table_2.columns =  ['title','price','amount']

In [10]:
table_2.to_csv('output.csv', index = False, encoding = 'utf-8-sig')
